In [39]:
import pandas as pd
from pandarallel import pandarallel
import numpy as np
from tqdm import tqdm
import datetime

from rectools.dataset.interactions import Interactions
from rectools.models.popular import PopularModel
from rectools.model_selection.time_split import TimeRangeSplitter
from rectools.dataset import Dataset
from rectools import Columns
from rectools.metrics import calc_metrics, Accuracy, NDCG

In [2]:
tqdm.pandas()
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 6 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


### Загрузка, знакомство, подготовка

**Взаимодействия пользователей с фильмами**

In [3]:
data = pd.read_csv('ml-latest/ratings.csv')

In [4]:
data.sample(5)

,userId,movieId,rating,timestamp
9581688,94221,2411,5.0,1279043298
5427593,52573,2762,4.0,939797939
7510398,73500,1302,3.0,854525600
21189963,206921,1036,3.0,954210491
20051737,196006,6539,5.0,1346836278


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33832162 entries, 0 to 33832161
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 1.0 GB


In [6]:
print(f'Испльзовано памяти: {data.memory_usage(deep=True).sum() / 1024 / 1024:.2f}mb')

Испльзовано памяти: 1032.48mb


In [7]:
#оптимизация хранения данных
data['userId'] = data['userId'].astype('int32')
data['movieId'] = data['movieId'].astype('int32')
data['rating'] = data['rating'].astype('float16')
#изменение хранения дат 
data['timestamp'] = pd.to_datetime(data['timestamp'].parallel_apply(lambda x: pd.Timestamp(x, unit='s').date()))

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33832162 entries, 0 to 33832161
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   userId     int32         
 1   movieId    int32         
 2   rating     float16       
 3   timestamp  datetime64[ns]
dtypes: datetime64[ns](1), float16(1), int32(2)
memory usage: 580.8 MB


In [9]:
print(f'Испльзовано памяти: {data.memory_usage(deep=True).sum() / 1024 / 1024:.2f}mb')

Испльзовано памяти: 580.77mb


**Названия фильмов и imbdID**

In [10]:
movies_ml = pd.read_csv('ml-latest/movies.csv')

In [11]:
movies_ml.sample(5)

,movieId,title,genres
52315,183579,Airwolf: The Movie (1984),Action|Adventure|Sci-Fi|Thriller
9633,31894,"ChubbChubbs!, The (2002)",Animation|Comedy|Sci-Fi
46096,170583,The Birds II: Land's End (1994),Horror
27741,129769,Covert Action (1978),(no genres listed)
30440,135833,One-Armed Boxer (1972),Action|Adventure


In [12]:
links_ml = pd.read_csv('ml-latest/links.csv')

In [13]:
links_ml.sample(5)

,movieId,imdbId,tmdbId
16800,88466,783695,22093.0
48705,175993,1771636,199887.0
34577,145072,783494,46526.0
12583,61037,841925,2012.0
18746,97880,56930,88912.0


## Постановка задачи и baseline

Используя историю взаимодействий пользователей с объектами создать двухэтапную рекомендательную модель, которая значительно превзойдёт базовую по метрике Serendipity (способность удивлять непопулярными релевантными объектами) на валидации, а также удовлетворит меня в ходе тестирования. 

Валидировать будем на данных последней недели по дням. В качестве бейзлайна возьмём popularity based модель, которая будет советовать самые популярные (больше всего пользователей посмотрело). Реализовывать первый этап и кросс-валидацию будем средствами библиотеки RecTools (https://github.com/MobileTeleSystems/RecTools)

In [65]:
data.columns = [Columns.User, Columns.Item, Columns.Weight, Columns.Datetime]
dataset = Dataset.construct(data)

In [59]:
cv = TimeRangeSplitter(test_size='1D', n_splits=7, 
                      filter_cold_users=False,
                      filter_cold_items=False)

In [81]:
for train_ids, test_ids, _ in cv.split(Interactions(data)):
    print(f"Max train date: {data.loc[train_ids]['datetime'].max()}")
    print(f"Max test date: {data.loc[test_ids]['datetime'].max()}")
    print('***' * 5)    

Max train date: 2023-07-13 00:00:00
Max test date: 2023-07-14 00:00:00
***************
Max train date: 2023-07-14 00:00:00
Max test date: 2023-07-15 00:00:00
***************
Max train date: 2023-07-15 00:00:00
Max test date: 2023-07-16 00:00:00
***************
Max train date: 2023-07-16 00:00:00
Max test date: 2023-07-17 00:00:00
***************
Max train date: 2023-07-17 00:00:00
Max test date: 2023-07-18 00:00:00
***************
Max train date: 2023-07-18 00:00:00
Max test date: 2023-07-19 00:00:00
***************
Max train date: 2023-07-19 00:00:00
Max test date: 2023-07-20 00:00:00
***************


In [43]:
baseline_model = PopularModel(period=datetime.timedelta(days=60),
                              popularity='n_users')

In [84]:
baseline_model.fit(Dataset.construct(data.loc[train_ids, :]))

In [93]:
baseline_model.recommend(users=data.loc[test_ids, Columns.User].unique(),
                        dataset=Dataset.construct(pd.concat([data.loc[train_ids, :], data.loc[test_ids, :]])),
                        k=10,
                        filter_viewed=True)

,user_id,item_id,score,rank
0,304,286897,523.0,1
1,304,283873,344.0,2
2,304,281096,329.0,3
3,304,134130,322.0,4
4,304,593,322.0,5
...,...,...,...,...
1125,329954,5952,401.0,6
1126,329954,285593,386.0,7
1127,329954,60069,370.0,8
1128,329954,68954,332.0,9
